In [1]:
import os 
kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
spark = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{} pyspark-shell".format(kafka, spark))

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
KAFKA_BROKER = "kafka-container:9092"
KAFKA_TOPIC = "item-shopee"
spark = SparkSession.builder.appName("shopee_analysis").master("spark://master:7077").getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-8_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3aab90b-33fd-4c7b-b53b-6fc6114b3b52;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found org.apache.spark#spark-streaming-kafka-0-8_2.11;2.1.0 in central
	found org.apache.kafka#kafka_2.11;0.8.2.1 in central
	found org.scala-lang.modules#scala-xml_2.11;1.0.2 in central
	found com.yammer.metrics#metrics-core;2.2.0 in central
	found org.scala-lang.modules#scala-parser-combinators_2.11;1.0.2 in central
	found com.101tec#zkclient;0.3 in central
	found log4j#log4j;1.2.17 in central
:: resolution report :: resolve 443ms :: artifacts dl 10ms
	:: modules in use:
	com.101tec#zkc

21/12/28 11:59:30 WARN TransportClientFactory: DNS resolution failed for master:7077 took 10008 ms
21/12/28 11:59:30 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master master:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:101)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:109)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:89)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:603)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngi

In [15]:
kafkaMessages = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("subscribe", KAFKA_TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()

In [16]:
message = kafkaMessages.selectExpr("CAST(value AS STRING)")

In [17]:
rawQuery = message \
        .writeStream \
        .queryName("qraw2")\
        .format("memory")\
        .start()

21/12/28 11:55:57 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b7e935b8-18dd-4048-bad7-909635d1bad5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/28 11:55:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
21/12/28 11:55:58 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0-1, groupId=spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
21/12/28 11:55:58 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-d

21/12/28 11:56:08 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0-1, groupId=spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
21/12/28 11:56:08 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0-1, groupId=spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected
21/12/28 11:56:09 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0-1, groupId=spark-kafka-source-ed5a8a7e-8103-4fa8-8f95-ab98d15a0adf--2037279542-driver-0] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
21/12/28 11:56:09 WARN NetworkC